In [ ]:
# REFACTOR: Standard imports + asyncio and nest_asyncio for Jupyter compatibility.
import pandas as pd
import numpy as np
import time
import datetime
import importlib
import inspect
import os
import re
import json
import random
import openai
import google.generativeai as genai
import anthropic
import asyncio
import nest_asyncio
from openai import AsyncOpenAI # REFACTOR: Import async client
from anthropic import AsyncClient # REFACTOR: Import async client
from datasets import load_dataset
from tqdm.notebook import tqdm # REFACTOR: Use notebook-friendly tqdm
from dotenv import load_dotenv
from typing import List, Dict, Any
from pathlib import Path
import textwrap, hashlib

def find_project_root():
    """Traverse upwards to find the project root, marked by the .git folder."""
    current_path = Path.cwd()
    while current_path != current_path.parent:
        if (current_path / ".git").is_dir():
            return current_path
        current_path = current_path.parent
    raise FileNotFoundError("Could not find project root. Is this a git repository?")

# REFACTOR: Apply nest_asyncio to allow asyncio to run in a Jupyter notebook.
# This must be done once per kernel.
nest_asyncio.apply()

# --- 1. Client and Model Configuration ---

load_dotenv()

# REFACTOR: Initialize asynchronous clients.
# The synchronous clients are no longer needed for the generation script.
openai_client_async = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
anthropic_client_async = AsyncClient(api_key=os.getenv("ANTHROPIC_API_KEY"))
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Define the project root as a global constant
PROJECT_ROOT = find_project_root()
print(f"Project root identified: {PROJECT_ROOT}")
BASE_OUTPUT_DIR = PROJECT_ROOT / 'data' / 'code_gen_outputs_raw'

# A safe starting point is slightly below the documented RPM.
API_CONCURRENCY_LIMITS = {
    "openai": 5,    # limit to 5 concurrent requests for OpenAI
    "anthropic": 5, # limit to 5 concurrent requests for Anthropic
    "google": 5,    # limit to 5 concurrent requests for Google
}

# --- 2. System Prompt and Helper Functions (no changes needed) ---

SYSTEM_PROMPT = "You are an expert Python programmer specializing in data formalization. Your role is to meticulously convert natural language math problems and their step-by-step solutions into a single, well-structured Python function. You will be presented with examples of the required format followed by a final task to complete."


PROMPT_GUIDELINES = """### Guidelines

0. **Output wrapping**
   Return the code inside a single ```python … ``` block, and nothing else.

1.  **Function Naming & Docstring:** The function must be named `solve`. It must begin with a docstring that has exactly two lines:
    *   The first line must be: "Index: [Index]." using the index from the task header.
    *   The second line must be a succinct, one-sentence description of what the function returns (e.g., "Returns: the total cost of wages and taxes.").

2.  **Function Arguments:** The function arguments must be derived from the 'Question' text. 
    *   Create a distinct argument for every numerical value that is directly stated in the text.
    *   The arguments should be created **in the same order in which they appear in the question**.
    *   **Note:** Some of these arguments may end up not being used in the function body. This is expected. Do not worry about this and leave the unused arguments in the function signature.

3.  **Argument Formatting:** Each argument must include a type-hint (e.g., `int`, `float`) and a default value equal to its value in the 'Question'. You must also add a comment (`#`) next to each argument that quotes or refers to the phrase in the 'Question' it comes from. 

4.  **Function Body:** The body of the function should follow the logic of the provided 'Solution' dict, which contains the step-by-step solution to the problem. The keys of this dict are strings (e.g. `"L1"`, `"L2"`) which refer to the line number, and the values of the dict are the corresponding steps in the solution. 
    * For every relevant line in the 'Solution', you must include a comment in the Python code that indicates the line number (key) from the 'Solution' dict.
    * These comments should be formatted as `#: L<n>`, where `<n>` is the line number from the 'Solution' dict.
    * Immediately follow the comment with the Python statement that performs the calculation.
    * Steps in the solution should result in the creation of new, intermediate variables, which should be named descriptively based on the context of the calculation.
    * Wherever possible, in your code try to use only the variables from the function arguments and the intermediate variables you created before, and try to avoid using hard-coded numbers in the calculations.

5.  **Calculator Annotations:** Pay close attention to the calculator annotations (e.g., `[[25*8=200]]`) in the 'Solution' as they reveal the precise mathematical operations to implement. **Note**: Some lines in the solution may not contain calculator annotations, but you should still pay attention to the logic and calculations described in those lines.

6.  **Final Answer:** Store the final answer in a variable named 'answer', and on the same line, add the comment `# FINAL ANSWER`. In the next line, return the 'answer' variable.

7. **No extra output:** Your output should end with the ``` closing the code block. Do not include any additional text, explanations, or comments outside of the code block."""

gsm8k_train = load_dataset("gsm8k", "main", split="train")

def build_solution_mapping(
    index: int,
    dataset: Any,
    convert_brackets: bool = True,
):
    """
    Parameters
    ----------
    index : int
        Position of the sample in the loaded dataset.
    dataset : iterable / HuggingFace Dataset
    convert_brackets : bool, default ``True``
        If ``True`` replace every ``<< … >>`` calculator annotation with
        the canonical ``[[ … ]]`` form so downstream code sees a single
        bracket style.

    Returns
    -------
    Dict[str, str]
        Mapping ``{"L1": <first non-empty line>, "L2": <second>, …}``.

    Notes
    -----
    * Blank lines in ``sample["answer"]`` are ignored.
    * The line numbering reflects the *order* in the original solution
      string; there is no semantic grouping beyond that.
    """
    # extract & split solution text
    solution_text = dataset[index]["answer"]
    lines = [ln.strip() for ln in solution_text.splitlines() if ln.strip()]

    # Remove the last line if it matches the '####' answer pattern
    if lines and re.match(r"^####\s*\d+(\.\d+)?$", lines[-1]):
        lines = lines[:-1]

    # optional bracket normalisation
    if convert_brackets:
        angle = re.compile(r"<<([^>]+)>>")
        lines = [angle.sub(r"[[\1]]", ln) for ln in lines]
    # build mapping
    return {f"L{i}": line for i, line in enumerate(lines, 1)}


def format_prompt_query(index: int, code_strings: dict, with_code: bool = False):
    sample = gsm8k_train[index]
    question = sample["question"]
    solution_mapping = build_solution_mapping(index, gsm8k_train)
    solution = json.dumps(solution_mapping)
    out = f"""*Index*: 
{index}

*Question*: 
{question}

*Solution*: 
{solution}

*Code*:"""
    if with_code:
        out += f"""\n```python
{code_strings.get(index, "# Code not found")}
```"""
    return out


def get_code_strings(indices: List[int], savepath: Path = PROJECT_ROOT / 'data' / 'code_examples'):
    """
    Reads code examples directly from .py files instead of importing them.
    This is more robust and avoids Python's complex import path mechanics.
    """
    code_strings = {}
    for idx in indices:
        # Construct the full file path
        filepath = os.path.join(savepath, f"_{idx}.py")
        try:
            # Read the entire content of the file
            with open(filepath, 'r', encoding='utf-8') as f:
                code_strings[idx] = f.read()
        except FileNotFoundError:
            print(f"Error: Could not find example file for index {idx} at: {filepath}")
            code_strings[idx] = f"# Error: Code for example {idx} not found."
    return code_strings


EXAMPLE_INDICES = [310, 3822, 7371]          # keep fixed order
_CODE_EXAMPLES  = get_code_strings(EXAMPLE_INDICES)


def build_static_prefix() -> str:
    """Guidelines + few-shot examples rendered once for caching."""
    examples_block = "\n".join(
        format_prompt_query(idx, _CODE_EXAMPLES, with_code=True)
        for idx in EXAMPLE_INDICES
    )
    prefix = "\n".join([
        PROMPT_GUIDELINES.strip(),
        "\n--- EXAMPLES ---\n",
        examples_block.strip()
    ])
    # Canonical whitespace – important for cache hits
    return textwrap.dedent(prefix).strip()


STATIC_PREFIX = build_static_prefix()
print(STATIC_PREFIX)
print("Static prefix SHA-1:", hashlib.sha1(STATIC_PREFIX.encode()).hexdigest())


def build_task_prompt(index: int) -> str:
    """Problem-specific part that changes every call."""
    return "\n".join([
        "--- TASK ---\n",
        format_prompt_query(index=index,
                            code_strings={},     # no code in the task part
                            with_code=False)
    ])


# --- 3. Asynchronous API Calling Function ---

async def call_model_api_async(
        provider: str,
        model: str,
        static_prefix: str,
        task_prompt: str,
        semaphore: asyncio.Semaphore):

    async with semaphore:
        usage = {"input_tokens": 0,
                 "output_tokens": 0,
                 "cached_tokens": 0}

        # ─────────────────── Google (no prompt cache) ───────────────────
        if provider == "google":
            gemini = genai.GenerativeModel(
                model_name=model,
                system_instruction=static_prefix,
            )
            cfg = genai.types.GenerationConfig(temperature=0.1, max_output_tokens=4000)
            response = await gemini.generate_content_async(task_prompt, generation_config=cfg)
            text = response.text
            if response.usage_metadata:
                usage["input_tokens"]  = response.usage_metadata.prompt_token_count
                usage["output_tokens"] = response.usage_metadata.candidates_token_count

        # ─────────────────── Anthropic (cache_control) ──────────────────
        elif provider == "anthropic":
            system_block = {
                "type": "text",
                "text": static_prefix,
                "cache_control": {"type": "ephemeral"},
            }
            response = await anthropic_client_async.messages.create(
                model=model,
                max_tokens=4000,
                temperature=0.1,
                system=[system_block],
                messages=[{"role": "user", "content": task_prompt}],
            )
            text = response.content[0].text
            usage["input_tokens"]   = response.usage.input_tokens
            usage["output_tokens"]  = response.usage.output_tokens
            usage["cached_tokens"]  = response.usage.cache_read_input_tokens

        # ─────────────────── OpenAI (auto prompt cache) ─────────────────
        elif provider == "openai":
            msgs = [
                {"role": "system", "content": static_prefix},
                {"role": "user",   "content": task_prompt},
            ]
            rsp = await openai_client_async.chat.completions.create(
                model=model,
                messages=msgs,
                temperature=0.1,
                max_tokens=4000,
            )
            text = rsp.choices[0].message.content
            usage["input_tokens"]  = rsp.usage.prompt_tokens
            usage["output_tokens"] = rsp.usage.completion_tokens
            try:
                details = rsp.usage.prompt_tokens_details
                usage["cached_tokens"] = details.get("cached_tokens", 0)
            except AttributeError:
                usage["cached_tokens"] = 0

        else:
            raise ValueError(f"Unknown provider {provider}")

        return text, usage


# --- 4. Parallel Orchestration Function ---

# ────────────────────────────────────────────────────────────────────
#  Parallel across PROBLEMS, but seed Anthropic cache with the first
#  problem before launching the rest.
# ────────────────────────────────────────────────────────────────────
async def generate_GSM8K_code_parallel(
    model_dict: Dict[str, List[str]],
    indices_to_generate: List[int],
    example_indices: List[int],
    output_dir: Path = BASE_OUTPUT_DIR,
    max_parallel_problems: int | None = None,      # None = unlimited
) -> pd.DataFrame:
    # choose a unique run-id right at the start  ────────────────────────
    run_ts = datetime.datetime.now(datetime.timezone.utc).strftime("%Y%m%d_%H%M%S")
    output_dir.mkdir(parents=True, exist_ok=True)
    provider_sems = {
        prov: asyncio.Semaphore(limit) for prov, limit in API_CONCURRENCY_LIMITS.items()
    }
    problem_sem = (
        asyncio.Semaphore(max_parallel_problems)
        if max_parallel_problems is not None
        else None
    )

    static_prefix = STATIC_PREFIX
    rows: list[dict] = []

    # ── coroutine for a single GSM8K problem ──────────────────────────
    async def _run_one_problem(idx: int):
        async def _guard():
            nonlocal rows
            task_prompt = build_task_prompt(idx)
            p_dir = output_dir / str(idx)
            p_dir.mkdir(parents=True, exist_ok=True)

            # launch model calls (your inner gather)
            tasks = []
            for prov, models in model_dict.items():
                for m in models:
                    t = asyncio.create_task(
                        call_model_api_async(
                            provider=prov,
                            model=m,
                            static_prefix=static_prefix,
                            task_prompt=task_prompt,
                            semaphore=provider_sems[prov],
                        )
                    )
                    t.meta = {"provider": prov, "model": m, "index": idx, "start": time.time()}
                    tasks.append(t)

            results = await asyncio.gather(*tasks, return_exceptions=True)
            for t, res in zip(tasks, results):
                meta = t.meta
                elapsed = time.time() - meta["start"]
                status = "Failed"
                text = ""
                usage = {"input_tokens": 0, "output_tokens": 0, "cached_tokens": 0}

                if not isinstance(res, Exception):
                    text, usage = res
                    status = "Success"
                    if text:
                        (p_dir / f"{meta['provider']}_{meta['model']}.txt").write_text(
                            text, encoding="utf-8"
                        )
                else:
                    print(f"{meta['provider']}_{meta['model']} ❌ {res}")

                rows.append(
                    {
                        "provider": meta["provider"],
                        "model": meta["model"],
                        "index": meta["index"],
                        "status": status,
                        "time_s": round(elapsed, 3),
                        "utc_completed": datetime.datetime.utcnow().isoformat(timespec="seconds"),
                        "input_tokens": usage["input_tokens"],
                        "output_tokens": usage["output_tokens"],
                        "cached_tokens": usage["cached_tokens"],
                    }
                )

        return await _guard() if problem_sem is None else await problem_sem.__aenter__() or await _guard() or await problem_sem.__aexit__(None, None, None)

    # ── 1. Seed cache with the FIRST index (blocking) ─────────────────
    first_idx, *rest = indices_to_generate
    await _run_one_problem(first_idx)

    # ── 2. Fan-out remaining problems concurrently ────────────────────
    await asyncio.gather(*[_run_one_problem(i) for i in rest])

    # ── save CSV ─────────────────────────────────────────────────────
    df = pd.DataFrame(rows)
    csv_path = output_dir / "generation_performance.csv"
    df.to_csv(csv_path, index=False)
    print(f"✓ All done — log at {csv_path}")
    return df

# # Add any problem indices you have generated outputs for.
# problem_indices_to_test = sorted([3331, 1647, 636, 399, 4670, 5918, 1531, 7364, 5464, 1205, 3518, 6732, 3779, 4483, 6237, 1202, 2345])

/opt/miniconda3/envs/erdos-dl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Project root identified: /Users/arvindsuresh/Documents/Github/Erdos-DL-June25-Math
### Guidelines

0. **Output wrapping**
   Return the code inside a single ```python … ``` block, and nothing else.

1.  **Function Naming & Docstring:** The function must be named `solve`. It must begin with a docstring that has exactly two lines:
    *   The first line must be: "Index: [Index]." using the index from the task header.
    *   The second line must be a succinct, one-sentence description of what the function returns (e.g., "Returns: the total cost of wages and taxes.").

2.  **Function Arguments:** The function arguments must be derived from the 'Question' text. 
    *   Create a distinct argument for every numerical value that is directly stated in the text.
    *   The arguments should be created **in the same order in which they appear in the question**.
    *   **Note:** Some of these arguments may end up not being used in the function body. This is expected. Do not worry about this and

In [2]:
# # ╔═══════════════════════════════════════════════════════════════════════╗
# # ║  Vertex-AI Gemini - Static-Prefix Context Cache (75 % input discount) ║
# # ╚═══════════════════════════════════════════════════════════════════════╝
# import os, datetime
# import vertexai
# from vertexai.generative_models import Content, GenerativeModel

# # ── 1. project / location  ──────────────────────────────────────────────
# PROJECT_ID = os.getenv("GOOGLE_PROJECT_ID")          # you just added this
# LOCATION   = "us-central1"                           # or "us-east4", etc.

# # If ADC is in place (`gcloud auth application-default login`) you can omit creds.
# vertexai.init(project=PROJECT_ID, location=LOCATION)

# # ── 2. Choose the model version you’ll call later  ──────────────────────
# MODEL_FOR_CACHE = "gemini-2.0-flash-thinking-exp"    # must match future calls

# # ── 3.  Cache identifiers  ──────────────────────────────────────────────
# CACHE_DISPLAY  = "gsm8k-static-1"                    # human-readable name
# GEMINI_CACHE_ID = (
#     f"projects/{PROJECT_ID}/locations/{LOCATION}/cachedContents/{CACHE_DISPLAY}"
# )

# # ── 4. Import the caching helper (path changed after 1.70)  ─────────────
# try:
#     from vertexai.preview import caching            # ≥ 1.70
# except ImportError:
#     raise ImportError(
#         "Your vertexai SDK is too old — please `pip install -U google-cloud-aiplatform`"
#     )

# # ── 5. Create the cache if it doesn’t exist  ────────────────────────────
# TTL_SECONDS = 60 * 60                               # 1 hour
# try:
#     cached = caching.CachedContent(cached_content_name=GEMINI_CACHE_ID)
#     cached.refresh()                                # raises if not found
#     print("🟢 Context cache already exists:", GEMINI_CACHE_ID)
# except Exception:
#     cached = caching.CachedContent.create(
#         model_name       = MODEL_FOR_CACHE,
#         system_instruction = STATIC_PREFIX,         # the long, static prefix
#         contents         = [],                      # we cache only the system prompt
#         ttl              = datetime.timedelta(seconds=TTL_SECONDS),
#         display_name     = CACHE_DISPLAY,
#     )
#     GEMINI_CACHE_ID = cached.name                   # update in case the API rewrites it
#     print("✅ Created context cache:", GEMINI_CACHE_ID)

# # ── 6. Helper to build the Content list in your async call  ─────────────
# def gemini_prompt_with_cache(task_prompt: str):
#     """Return the list of Content objects that prepends cached tokens."""
#     return [
#         Content(cached_content=GEMINI_CACHE_ID),    # prepended, billed at 25 %
#         Content(role="user", text=task_prompt),     # dynamic part
#     ]

In [ ]:
# --- 5. Execution ---

# Define your parameters here
indices = [310, 3822, 7371] # Use the indices of your few-shot examples
indices_to_generate = list(range(50))

model_dict = {
  "anthropic": ["claude-3-5-haiku-20241022"], 
  "openai": ["gpt-4.1-mini"],
  "google": ["gemini-2.0-flash-thinking-exp", 
             "gemini-2.5-flash-lite-preview-06-17",
             "gemini-2.5-flash"]
}

# REFACTOR: To run the async function, you must `await` it.
# This will execute the entire parallel generation process.
# The result (a pandas DataFrame with performance logs) will be stored in `perf_df`.

perf_df = await generate_GSM8K_code_parallel(
    model_dict=model_dict,
    indices_to_generate=indices_to_generate,
    example_indices=indices
)

# print("\nFinal Performance Summary:")
# print(perf_df)

I0000 00:00:1750728541.409956 7935654 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
/var/folders/yk/x70ww_h95d3fgd5spjp1s0780000gn/T/ipykernel_3553/3957948032.py:364: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "utc_completed": datetime.datetime.utcnow().isoformat(timespec="seconds"),


✓ All done — log at /Users/arvindsuresh/Documents/Github/Erdos-DL-June25-Math/data/code_gen_outputs_raw/generation_performance.csv


In [4]:
perf_df

,provider,model,index,status,time_s,utc_completed,input_tokens,output_tokens,cached_tokens
0,anthropic,claude-3-5-haiku-20241022,0,Success,3.377,2025-06-24T01:29:04,138,175,3428
1,openai,gpt-4.1-mini,0,Success,3.378,2025-06-24T01:29:04,3162,120,3072
2,google,gemini-2.0-flash-thinking-exp,0,Success,3.379,2025-06-24T01:29:04,3600,149,0
3,google,gemini-2.5-flash-lite-preview-06-17,0,Success,3.380,2025-06-24T01:29:04,3600,137,0
4,google,gemini-2.5-flash,0,Success,3.380,2025-06-24T01:29:04,3600,125,0
5,anthropic,claude-3-5-haiku-20241022,1,Success,2.432,2025-06-24T01:29:07,142,142,3428
6,openai,gpt-4.1-mini,1,Success,2.433,2025-06-24T01:29:07,3163,121,3072
7,google,gemini-2.0-flash-thinking-exp,1,Success,2.433,2025-06-24T01:29:07,3601,132,0
8,google,gemini-2.5-flash-lite-preview-06-17,1,Success,2.433,2025-06-24T01:29:07,3601,128,0
9,google,gemini-2.5-flash,1,Success,2.433,2025-06-24T01:29:07,3601,133,0
